In [1]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 1 : Imports et Configuration
# ════════════════════════════════════════════════════════════════════

import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv('../config/.env')

print("✅ Imports OK")

✅ Imports OK


In [2]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 2 : Vérification de la Connexion RDS
# ════════════════════════════════════════════════════════════════════

print("🔗 VÉRIFICATION DE LA CONNEXION RDS\n")
print("="*60)

# Configuration
RDS_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'port': int(os.getenv('DB_PORT', 5432)),
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# Afficher la configuration (masquer le mot de passe)
print("📋 Configuration RDS :")
for key, value in RDS_CONFIG.items():
    if key == 'password':
        print(f"   • {key:12s} : {'*' * 8}")
    else:
        print(f"   • {key:12s} : {value}")

print("\n🔗 Test de connexion...")

try:
    # Connexion
    conn = psycopg2.connect(**RDS_CONFIG)
    cursor = conn.cursor()
    
    print("✅ Connexion RDS réussie !\n")
    
    # Version PostgreSQL
    cursor.execute("SELECT version();")
    version = cursor.fetchone()[0]
    print(f"📊 PostgreSQL : {version.split(',')[0]}")
    
    # Taille de la base
    cursor.execute("""
        SELECT pg_size_pretty(pg_database_size(current_database())) as size;
    """)
    db_size = cursor.fetchone()[0]
    print(f"💾 Taille de la base : {db_size}")
    
    # Connexions actives
    cursor.execute("""
        SELECT count(*) FROM pg_stat_activity 
        WHERE datname = current_database();
    """)
    connections = cursor.fetchone()[0]
    print(f"🔌 Connexions actives : {connections}")
    
    # Tables existantes
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    tables = cursor.fetchall()
    
    if tables:
        print(f"\n📊 Tables existantes ({len(tables)}) :")
        for table in tables:
            cursor.execute(sql.SQL("SELECT COUNT(*) FROM {};").format(
                sql.Identifier(table[0])
            ))
            count = cursor.fetchone()[0]
            print(f"   • {table[0]:30s} ({count:,} lignes)")
    else:
        print("\n📭 Aucune table dans la base de données")
        print("   Les tables seront créées dans la prochaine cellule")
    
    cursor.close()
    conn.close()

except psycopg2.OperationalError as e:
    print(f"❌ Erreur de connexion : {e}\n")
    print("💡 Vérifications :")
    print("   1. L'instance RDS est démarrée")
    print("   2. Le endpoint (RDS_HOST) est correct")
    print("   3. Le groupe de sécurité autorise votre IP")
    print("   4. Le mot de passe est correct")

except psycopg2.Error as e:
    print(f"❌ Erreur PostgreSQL : {e}")

🔗 VÉRIFICATION DE LA CONNEXION RDS

📋 Configuration RDS :
   • host         : kayak-db.cvumwuoaepex.eu-west-3.rds.amazonaws.com
   • port         : 5432
   • database     : kayak-db
   • user         : kayakdb
   • password     : ********

🔗 Test de connexion...
✅ Connexion RDS réussie !

📊 PostgreSQL : PostgreSQL 17.4 on aarch64-unknown-linux-gnu
💾 Taille de la base : 7795 kB
🔌 Connexions actives : 1

📭 Aucune table dans la base de données
   Les tables seront créées dans la prochaine cellule


In [3]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 3 : Création du Schéma de Base de Données
# ════════════════════════════════════════════════════════════════════

print("📋 CRÉATION DU SCHÉMA DE BASE DE DONNÉES\n")
print("="*60)

# SQL de création des tables
create_tables_sql = """
-- Supprimer les tables existantes (si elles existent)
DROP TABLE IF EXISTS recommendations CASCADE;
DROP TABLE IF EXISTS weather_history CASCADE;
DROP TABLE IF EXISTS hotels CASCADE;
DROP TABLE IF EXISTS cities CASCADE;

-- Table des villes
CREATE TABLE cities (
    city_id SERIAL PRIMARY KEY,
    city_name VARCHAR(100) NOT NULL UNIQUE,
    latitude DECIMAL(10,8),
    longitude DECIMAL(11,8),
    avg_weather_score DECIMAL(5,2),
    temp_avg DECIMAL(5,2),
    rain DECIMAL(5,2),
    humidity DECIMAL(5,2),
    wind_speed DECIMAL(5,2),
    clouds DECIMAL(5,2),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Table des données météo historiques
CREATE TABLE weather_history (
    weather_id SERIAL PRIMARY KEY,
    city_id INTEGER REFERENCES cities(city_id) ON DELETE CASCADE,
    date_recorded DATE NOT NULL,
    temp_avg DECIMAL(5,2),
    precipitation DECIMAL(5,2),
    sun_hours DECIMAL(5,2),
    wind_speed DECIMAL(5,2),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Table des hôtels
CREATE TABLE hotels (
    hotel_id SERIAL PRIMARY KEY,
    city_id INTEGER REFERENCES cities(city_id) ON DELETE CASCADE,
    hotel_name VARCHAR(255) NOT NULL,
    url TEXT,
    property_type VARCHAR(100),
    score DECIMAL(3,1),
    number_of_reviews INTEGER,
    price DECIMAL(10,2),
    currency VARCHAR(10),
    latitude DECIMAL(10,8),
    longitude DECIMAL(11,8),
    facilities TEXT,
    image_url TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Table des recommandations finales
CREATE TABLE recommendations (
    rec_id SERIAL PRIMARY KEY,
    hotel_id INTEGER REFERENCES hotels(hotel_id) ON DELETE CASCADE,
    rank INTEGER NOT NULL,
    weather_score_norm DECIMAL(5,2),
    hotel_score_norm DECIMAL(5,2),
    price_score_norm DECIMAL(5,2),
    final_score DECIMAL(5,2),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UNIQUE(hotel_id)
);

-- Index pour améliorer les performances
CREATE INDEX idx_hotels_city ON hotels(city_id);
CREATE INDEX idx_hotels_score ON hotels(score DESC);
CREATE INDEX idx_hotels_price ON hotels(price);
CREATE INDEX idx_recommendations_rank ON recommendations(rank);
CREATE INDEX idx_recommendations_final_score ON recommendations(final_score DESC);
CREATE INDEX idx_weather_history_city_date ON weather_history(city_id, date_recorded);

-- Vue pour les meilleures recommandations
CREATE OR REPLACE VIEW top_recommendations AS
SELECT 
    r.rank,
    c.city_name,
    h.hotel_name,
    h.score as hotel_score,
    h.price,
    h.currency,
    h.number_of_reviews,
    c.avg_weather_score,
    c.temp_avg,
    r.final_score,
    h.url,
    h.latitude,
    h.longitude
FROM recommendations r
JOIN hotels h ON r.hotel_id = h.hotel_id
JOIN cities c ON h.city_id = c.city_id
ORDER BY r.rank;

-- Vue pour les statistiques par ville
CREATE OR REPLACE VIEW city_statistics AS
SELECT 
    c.city_name,
    c.avg_weather_score,
    c.temp_avg,
    c.rain,
    COUNT(h.hotel_id) as total_hotels,
    AVG(h.score) as avg_hotel_score,
    AVG(h.price) as avg_price,
    MIN(h.price) as min_price,
    MAX(h.price) as max_price,
    SUM(h.number_of_reviews) as total_reviews
FROM cities c
LEFT JOIN hotels h ON c.city_id = h.city_id
GROUP BY c.city_id, c.city_name, c.avg_weather_score, c.temp_avg, c.rain
ORDER BY c.avg_weather_score DESC;

-- Vue pour les meilleurs hôtels par ville
CREATE OR REPLACE VIEW best_hotels_by_city AS
SELECT 
    c.city_name,
    h.hotel_name,
    h.score,
    h.price,
    h.currency,
    h.number_of_reviews,
    r.final_score,
    r.rank
FROM hotels h
JOIN cities c ON h.city_id = c.city_id
LEFT JOIN recommendations r ON h.hotel_id = r.hotel_id
WHERE h.score IS NOT NULL
ORDER BY c.city_name, h.score DESC;
"""

print("⚙️  Création des tables et des vues...\n")

try:
    conn = psycopg2.connect(**RDS_CONFIG)
    cursor = conn.cursor()
    
    # Exécuter le script SQL
    cursor.execute(create_tables_sql)
    conn.commit()
    
    print("✅ Schéma créé avec succès !\n")
    
    # Vérifier les tables créées
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public' 
        AND table_type = 'BASE TABLE'
        ORDER BY table_name;
    """)
    tables = cursor.fetchall()
    
    print(f"📊 Tables créées ({len(tables)}) :")
    for table in tables:
        print(f"   • {table[0]}")
    
    # Vérifier les vues créées
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.views 
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    views = cursor.fetchall()
    
    print(f"\n📈 Vues créées ({len(views)}) :")
    for view in views:
        print(f"   • {view[0]}")
    
    cursor.close()
    conn.close()
    
except psycopg2.Error as e:
    print(f"❌ Erreur lors de la création du schéma : {e}")
    if conn:
        conn.rollback()

📋 CRÉATION DU SCHÉMA DE BASE DE DONNÉES

⚙️  Création des tables et des vues...

✅ Schéma créé avec succès !

📊 Tables créées (4) :
   • cities
   • hotels
   • recommendations
   • weather_history

📈 Vues créées (3) :
   • best_hotels_by_city
   • city_statistics
   • top_recommendations
